In [1]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/adoptopenjdk-16.jdk/Contents/Home'
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import numpy as np
import datetime
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('spark-job') \
    .config('spark.executor.memory', '4gb') \
    .config("spark.executor.cores", "4") \
    .getOrCreate()

24/08/24 18:09:21 WARN Utils: Your hostname, admins-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 172.30.1.45 instead (on interface en0)
24/08/24 18:09:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/24 18:09:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/24 18:09:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# TO BE UPDATED
POST_FP = "../EDA/gu/data/아이오닉 누수/clien_posts.csv" # from S3
COMMENT_FP = "../EDA/gu/data/아이오닉 누수/clien_comments.csv" # from S3
MODEL_FP = "../model_result.csv" # from RedShift
OUTPUT_FP = "output/transform_result" # to RedShift

In [3]:
def find_hot_criteria(row, model_df) -> np.float64:
    condition = ((model_df['post_type']==1) \
        & (model_df['relative_time']==row.relatvie_time) \
        & (model_df['cumulative_num']==row.comments))
    hot_pdf_value = model_df[condition].pdf.values[0]
    # print("find_hot_criteria", type(hot_pdf_value))
    return hot_pdf_value


def find_cold_criteria(row, model_df) -> np.float64:
    condition = ((model_df['post_type']==0) \
        & (model_df['relative_time']==row.relatvie_time) \
        & (model_df['cumulative_num']==row.comments))
    cold_pdf_value = model_df[condition].pdf.values[0]
    # print("find_cold_criteria", type(cold_pdf_value))
    return cold_pdf_value


def predict_post_type(post_df, model_df) -> ps.DataFrame:  
    per_post_hot_probability = post_df.apply(lambda x: find_hot_criteria(x, model_df), axis=1)
    per_post_cold_probability = post_df.apply(lambda x: find_cold_criteria(x, model_df), axis=1)
    post_df['impact'] = np.log(per_post_hot_probability/per_post_cold_probability).round(2)
    post_df['post_type'] = 0
    post_df.loc[post_df['impact'] > 0, 'post_type'] = 1    
    return post_df


def parse_date(date_str):
    return ps.to_datetime(date_str, errors="coerce")

In [4]:
post_df = spark.read.option("header", True).option("quote", "\"").option("escape", "\"").option("multiline", "true").option("sep", ",").csv(POST_FP, sep=',',).pandas_api()
comment_df = spark.read.option("header", True).option("quote", "\"").option("multiline", "true").csv(COMMENT_FP, sep=',').pandas_api() 
model_df = spark.read.option("header", True).csv(MODEL_FP, sep=',').pandas_api()

collected_at = ps.to_datetime('2023-10-31 00:00:52').replace(second=0, microsecond=0) # from S3 (crawling)
if collected_at.minute >= 30:
    collected_at = collected_at.replace(minute=30)
else:
    collected_at = collected_at.replace(minute=00)

/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(


In [5]:
post_df['created_at'] = post_df['created_at'].apply(parse_date)
comment_df.columns = ['cmt_author', 'cmt_content', 'post_id', 'cmt_created_at']     
comment_df['cmt_created_at'] = comment_df['cmt_created_at'].apply(parse_date)
model_df['pdf'] = model_df['pdf'].astype(float)
model_df['cumulative_num'] = model_df['cumulative_num'].astype(int)
model_df['post_type'] = model_df['post_type'].astype(int)
model_df['post_type'] = model_df['post_type'].astype(int)

/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(
/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(


In [6]:
# filtering (only for test)
cutoff_time = collected_at - datetime.timedelta(hours=12)
post_df = post_df[post_df['created_at'] > cutoff_time]
comment_df = comment_df[comment_df['cmt_created_at'] > cutoff_time]
post_df = post_df[post_df['created_at'] <= collected_at] 
comment_df = comment_df[comment_df['cmt_created_at'] <= collected_at]

In [7]:
number_of_comment_per_post_df = comment_df[['post_id', 'cmt_author']].groupby(['post_id']).agg({'cmt_author': 'count'}).rename(columns={'cmt_author': 'comments'})
number_of_comment_per_post_df = number_of_comment_per_post_df.reset_index()
number_of_comment_per_post_df.loc[number_of_comment_per_post_df['comments']>200] = 200

In [8]:
TIME_INTERVAL = 5 * 60
post_df = ps.merge(post_df, number_of_comment_per_post_df, left_on='id', right_on='post_id', how='left')
post_df = post_df.drop(columns=['id'])
post_df['relatvie_time'] =  ((collected_at - post_df['created_at']) // TIME_INTERVAL) * 5
post_df = predict_post_type(post_df, model_df)
post_df['sns_id'] = 0 # TODO: use sns_id from input data. (to be deleted)
post_df['collected_at'] = collected_at
post_df['batch_id'] = int(collected_at.strftime('%Y%m%d%H%M'))

/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/pandas/data_type_ops/datetime_ops.py:98: UserWarning: Note that there is a behavior difference of timestamp subtraction. The timestamp subtraction returns an integer in seconds, whereas pandas returns 'timedelta64[ns]'.
  warnings.warn(msg, UserWarning)
/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datet

In [9]:
post_df.to_csv(OUTPUT_FP, header=True)

/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
/Users/admin/Desktop/softeer-team-project/.final_check/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/namespace.py:1744: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is